In [1]:
import os
os.chdir("../../")
os.getcwd()

'/home/yeonwoo/code/study/langchain_note'

In [2]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
llm = ChatOpenAI(temperature=0)
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

#### 출력파서 사용하지 않는 경우

In [34]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

chain = prompt | llm
chain.invoke({"email_conversation": email_conversation}).content

'- ZENESIS 모델에 대한 상세한 브로슈어 요청\n- 유통 전략과 마케팅 계획을 구체화하기 위한 협력 논의\n- 다음 주 화요일(1월 15일) 오전 10시에 미팅 제안'

#### 출력파서 사용하는 경우

In [35]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [36]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"title": "Person", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\\uba54\\uc77c\\uc744 \\ubcf4\\ub0b8 \\uc0ac\\ub78c\\uc758 \\uc774\\uba54\\uc77c \\uc8fc\\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\\uba54\\uc77c \\uc81c\\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\\uba54\\uc77c \\ubcf8\\ubb38\\uc744 \\uc694\\uc57d\\ud55c \\ud14d\\uc2a4\\ud2b8", "type": "string

In [41]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())
# chain 을 생성합니다.
chain = prompt | llm 
output = chain.invoke({
    "email_conversation": email_conversation,
    "question": "이메일 내용중 주요 내용을 반드시 format 형식에 맞게 추출해 주세요.",
})

In [43]:
output.content

'{\n  "person": "김철수",\n  "email": "chulsoo.kim@bikecorporation.me",\n  "subject": "ZENESIS 자전거 유통 협력 및 미팅 일정 제안",\n  "summary": "ZENESIS 모델에 대한 상세한 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 측면에 대한 정보가 필요하다고 언급함. 또한, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안함.",\n  "date": "미팅 제안일자: 다음 주 화요일(1월 15일) 오전 10시"\n}'

In [44]:
import json
json.loads(output.content)

{'person': '김철수',
 'email': 'chulsoo.kim@bikecorporation.me',
 'subject': 'ZENESIS 자전거 유통 협력 및 미팅 일정 제안',
 'summary': 'ZENESIS 모델에 대한 상세한 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 측면에 대한 정보가 필요하다고 언급함. 또한, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안함.',
 'date': '미팅 제안일자: 다음 주 화요일(1월 15일) 오전 10시'}

In [45]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 반드시 format 형식에 맞게 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary='ZENESIS 모델에 대한 상세한 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 측면에 대한 정보 필요. 다음 주 화요일(1월 15일) 오전 10시에 미팅 제안.', date='2023-01-15 10:00 AM')

In [46]:
llm_with_structered = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(EmailSummary)
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 이은채 대리에게 ZENESIS 자전거에 대한 협력 제안과 미팅 일정을 제안하는 이메일을 보냈습니다. ZENESIS 모델에 대한 상세한 정보를 요청하고, 다음 주 화요일 오전 10시에 미팅을 제안했습니다.', date='다음 주 화요일(1월 15일) 오전 10시')